<a href="https://colab.research.google.com/github/TienNguyen0712/student-alcohol-consumption-data-mining/blob/main/notebooks/05_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🎯 **Problem Formulation (Định nghĩa mục tiêu)**

## 🔎 **Data Preparation (Chuẩn bị dữ liệu phân cụm)**

## 📐 **Optimal K (Tối ưu cụm K)**

## 🤖 **Clustering Model (Mô hình phân cụm)**

## 📊 **Visualization (Trực quan hóa)**

## 🧬 **Cluster Analysis (Phân tích cụm)**

## 📝 **Conclusion (Kết luận)**